In [1]:
from langchain_core.tools import tool

In [2]:
# s1 - create a function
def multiply(a, b):
    '''Multiplies two numbers together.'''
    return a*b

In [3]:
# s2 - add type hints (type hinting)
def multiply(a: int, b: int) -> int:
    '''Multiplies two numbers together.'''
    return a*b

##### M1: using tool decorator

In [4]:
# s3 - add tool decorator

"""
    @tool - decorator is used to mark this function as a special function.
"""
@tool
def multiply(a: int, b: int) -> int:
    '''Multiplies two numbers together.'''
    return a*b

In [7]:
ans = multiply.invoke({'a':5, 'b':8})
print(ans)

40


In [9]:
# properties

print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Multiplies two numbers together.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


##### M2: Using Structured tool

*It's a special type of tool where input follows a structured schema, defined using pydantic model*

In [1]:
from langchain.tools import StructuredTool
from pydantic import BaseModel, Field

In [ ]:
# pydantic class inherited from BaseModel
class MultiplyIntegers(BaseModel):
    a: int = Field(required=True, description="The first number to multiply.")
    b: int = Field(required=True, description="The second number to multiply.")


In [ ]:
# function to be used
def multiplyNums(a: int, b: int) -> int:
    return a*b

In [ ]:
# converting the function to a tool using StructuredTool
multiply_tool = StructuredTool.from_function(
    func= multiplyNums, # defined function name
    name="multiply", # name of tool
    description="Multiply two numbers", # description of tool
    args_schema=MultiplyIntegers # pydantic class containing information about the arguments
)

In [5]:
res = multiply_tool.invoke({"a": 5, "b": 8})

print(res)
print(multiply_tool.name)
print(multiply_tool.description)

40
multiply
Multiply two numbers


##### M3: using BaseTool Class

*BaseTool is the abstract base class for tools in langchain. It defines the core structure of tool. @tool is built on top of it.*

In [10]:
from langchain.tools import BaseTool
from typing import Type

In [11]:
# argument structuring using pydantic
class MultiplyIntegers(BaseModel):
    a: int = Field(required=True, description="The first number to multiply.")
    b: int = Field(required=True, description="The second number to multiply.")

In [ ]:
# tool class definition
class MultiplyTool(BaseTool):
    name: str = "multiply" # tool name
    description: str = "Multiply two numbers" # tool description
    
    args_schema: Type[BaseModel] = MultiplyIntegers # pydantic class containing information about the arguments
    
    def _run(self, a: int, b: int) -> int:
        return a*b

In [14]:
multiply_tool = MultiplyTool() # creating an instance of the tool class

In [17]:
res = multiply_tool.invoke({"a": 5, "b": 8})

print(res)
print(multiply_tool.name)
print(multiply_tool.description)
print(multiply_tool.args_schema.model_json_schema()) # schema of the arguments

40
multiply
Multiply two numbers
{'properties': {'a': {'description': 'The first number to multiply.', 'required': True, 'title': 'A', 'type': 'integer'}, 'b': {'description': 'The second number to multiply.', 'required': True, 'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'MultiplyIntegers', 'type': 'object'}


## Custom Toolkit

In [19]:
from langchain_core.tools import tool
@tool
def add(a: int, b: int) -> int:
    '''Adds two numbers together.'''
    return a + b

@tool
def subtract(a: int, b: int) -> int:
    '''Subtracts two numbers.'''
    return a - b


In [20]:
class MathsToolkit:
    def get_tools(self):
        return [add, subtract]

In [21]:
toolkit = MathsToolkit()

tools = toolkit.get_tools()

for tool in tools:
    print(tool.name)
    print(tool.description)
    print(tool.args_schema.model_json_schema()) # schema of the arguments

add
Adds two numbers together.
{'description': 'Adds two numbers together.', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'add', 'type': 'object'}
subtract
Subtracts two numbers.
{'description': 'Subtracts two numbers.', 'properties': {'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}, 'required': ['a', 'b'], 'title': 'subtract', 'type': 'object'}
